# Complexidade dos Testes

<span style="color:blue"> **Próximos Passos:** </span>

Sistemas Envolvidos - Montar nivel de Complexidade
- Tempo Execução
- Quantidade de Steps
- Combinação com o Sistema Principal
- Um Cenrário, todos os Casos de testes são feitos no mesmo sistema?
- Poderá ter vies do funcionario ter mais habilidade e por isso o tempo de execução ser menor?
- Faturável ou não faturável: Pode ter maior concentração de rejeição em algum sistema

In [ ]:
# Map the Dependencies
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import missingno as msno
import re

sns.set()

plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### 1. Import das bases de Execução

In [ ]:
exec_fat = pd.read_csv("data/Exec_Faturaveis.csv", 
                       parse_dates=['Data_Execucao', 'Data_FechamentoCenario', 'Hora_Execucao', 'Fechamento_CT'])
exec_nfat = pd.read_csv("data/Exec_NaoFaturaveis.csv",
                       parse_dates=['Data_Execucao', 'Data_FechamentoCenario', 'Hora_Execucao','Fechamento_CT'])
exec_fcst = pd.read_csv("data/Exec_Forecast.csv",
                       parse_dates=['Data_Execucao', 'Data_FechamentoCenario', 'Hora_Execucao', 'Fechamento_CT'])

In [ ]:
exec_full = pd.concat([exec_fat, exec_nfat, exec_fcst], ignore_index=True)
exec_full.tail()
exec_full.shape

In [ ]:
exec_full.columns
exec_full.shape

### 2. Filtra somente testes com tempo de execução

In [ ]:
# Filtra somente testes com tempo de execução
exec_full_valid = exec_full[exec_full["Duracao_Segundos"]>0]

In [ ]:
exec_full_valid.shape

In [ ]:
exec_full_valid.Nome_CT.unique()

### 3. Extraindo somente o verbo da Execução

In [ ]:
# Substituir os espaços " - " por "-"
exec_full_valid['Nome_CT'] = exec_full_valid['Nome_CT'].apply([lambda x: re.sub(r'\s+', ' ', x)])
exec_full_valid['Nome_CT'] = exec_full_valid['Nome_CT'].apply([lambda x: x.replace(r' - ', '-')])

In [ ]:
# Extrair somente o verbo o caso de teste
def verbo_CT (texto):
    a = texto.split(" ", 1)[0]
    a.split(' ', 1)
    word1 = " ".join(re.findall("[a-zA-Z]+", a))
    #verbo = word1.split(" ", 1)[1]
    return word1

def replace_CT (texto):
    verbo = texto.split(" ", 10)[1]
    return verbo

In [ ]:
# Tratando na mão casos bizarros
exec_full_valid['Verbo_test'] = exec_full_valid['Nome_CT'].apply(lambda x : verbo_CT(x))
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('CT', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C T ATLYS ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C T API ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C T NGIN ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('T ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('TC ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('CN ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C BDFIN ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C PUTTY ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C BDA ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('ATLYS ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C Putty ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C Atlys', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('C ', '')])
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace('PUTTY', '')])

In [ ]:
exec_full_valid['Verbo_test'] = exec_full_valid['Verbo_test'].apply([lambda x: x.replace(r' ', '')])

### 4. DataSet dos Sisitemas + Verbo

In [ ]:
complex_sist = exec_full_valid.groupby(['Sistemas_Envolvidos','Sistema_Principal','Verbo_test']).agg({'Testador_Real': {"qtde_total_teste": 'count'},
                                                                                   'Duracao_Segundos': {"tempo_medio": 'mean', "tempo_min": 'min', "tempo_max": 'max', "tempo_total": 'sum', "tempo_std": 'std'},
                                                                                   'Qtde_TotalSteps': {"qtde_steps_medio": 'mean',"qtde_steps_min": 'min',"qtde_steps_max": 'max',"qtde_steps_sum": 'sum', "qtde_steps_std": 'std'},
                                                                                   'Qtde_StepsATTACHMENT': {"qtde_steps_att_medio": 'mean',"qtde_steps_att_min": 'min',"qtde_steps_att_max": 'max',"qtde_steps_att_sum": 'sum',"qtde_steps_att_std": 'std'},
                                                                                   'Qtde_ExecPassed': {"qtde_steps_pass_medio": 'mean',"qtde_step_pass_min": 'min',"qtde_steps_pass_max": 'max',"qtde_steps_pass_sum": 'sum', "qtde_steps_pass_std": 'std'}})
                                                                                  
                                                                        
complex_sist.columns = complex_sist.columns.droplevel(0)
complex_sist = complex_sist.reset_index()
complex_sist.head(2)                                                                              

In [ ]:
complex_sist.shape

### 5. DataSet do tempo gasto de acordo com o VERBO

In [ ]:
verbo = exec_full_valid.groupby(['Verbo_test']).agg({'Duracao_Segundos': {"tempo_medio": 'mean', "tempo_min": 'min', "tempo_max": 'max'},
                                                     'Qtde_TotalSteps': {"qtde_steps_medio": 'mean',"qtde_steps_min": 'min',"qtde_steps_max": 'max'}})
                                                                                  
                                                                        
verbo.columns = verbo.columns.droplevel(0)
verbo = verbo.reset_index()
verbo.head()  

In [ ]:
verbo.shape

In [ ]:
verbo.describe().T

### 6. Combinando os dois DataSets e Extraindo as Dumies da combinação dos sistemas

In [ ]:
complex_sist = complex_sist.merge(verbo, left_on='Verbo_test', right_on='Verbo_test',suffixes=('', '_verbo'))
complex_sist.tail()

In [ ]:
sist_dummies = complex_sist.Sistema_Principal.str.get_dummies()

In [ ]:
sist_dummies.shape

In [ ]:
complex_sist.shape

In [ ]:
import missingno as msno
msno.matrix(complex_sist)

In [ ]:
#complex_sist_fim = pd.concat([sist_dummies, complex_sist], join="inner")
complex_sist_fim = complex_sist.merge(sist_dummies, left_on=sist_dummies.index, right_on=complex_sist.index)
complex_sist_fim.tail()

In [ ]:
complex_sist_fim.shape

In [ ]:
list(complex_sist_fim.columns)

In [ ]:
# Dropei estas columas por conterem missing 
complex_sist_fim = complex_sist_fim.drop(['key_0', "tempo_std", "qtde_steps_std", "qtde_steps_pass_std",'qtde_steps_att_std'], axis=1)

### 7. Cluster da Complexidade

In [ ]:
#Verifica presença de missing
import missingno as msno
msno.matrix(complex_sist_fim)

In [ ]:
complex_sist_fim['tempo_medio_step'] = complex_sist_fim['tempo_total']/complex_sist_fim['qtde_steps_sum']

In [ ]:
complex_sist_fim.describe().T

In [ ]:
# Dropei estas columas que se relacionam com tempo por step
columas_logit = complex_sist_fim.drop(["tempo_total", "qtde_steps_sum"], axis=1)

In [ ]:
columas_logit.head()

## <a id='1.1'> 8. Clustering Complexidade dos Testes por Sistema </a>
Voltar: <a href='#0'>Sumário  </a>

In [ ]:
columas_logit.info(verbose=True)

### 8. Clustering

In [ ]:
columas_logit = complex_sist_fim.drop(["tempo_total", "qtde_steps_sum"], axis=1)

In [ ]:
corr = complex_sist.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

with sns.axes_style("white"):
    fig, ax = plt.subplots(figsize=(15, 12));
    
    sns.heatmap(
        corr,
        ax=ax,
        annot=True,
        mask=mask,
        square=True
    );

In [ ]:
import statsmodels.formula.api as smf

def run_regression(df,function):
    model = smf.ols(function, df).fit()
    print(model.summary())

In [ ]:
run_regression(columas_logit,'''tempo_medio_step ~  qtde_total_teste + tempo_min + tempo_max''')

In [ ]:
list(complex_sist.columns)

### Clustering

In [ ]:
colunas_clustering = complex_sist.drop([   
"Sistema_Principal",
"Sistemas_Envolvidos",
"Verbo_test",   
"qtde_steps_att_sum",
"qtde_steps_att_max",
"tempo_medio_verbo",
"tempo_min_verbo",
"qtde_steps_min",
"qtde_steps_max",
"tempo_min",
"qtde_steps_pass_medio",
"qtde_steps_pass_max",
"tempo_std", 
"qtde_steps_std", 
"qtde_steps_pass_std",
'qtde_steps_att_std'
], axis=1)

In [ ]:
#Verifica presença de missing
import missingno as msno
msno.matrix(colunas_clustering)

In [ ]:
colunas_clustering.columns

In [ ]:
from sklearn.cluster import KMeans

plt.rcParams['figure.figsize'] = (12, 6)

def metodo_elbow(max_nro_clusters, X):
    wcss = [] 
    for i in range(1, max_nro_clusters):
        kmeans = KMeans(n_clusters = i, init = 'random')
        kmeans.fit(X)
        #print (i,kmeans.inertia_)
        wcss.append(kmeans.inertia_)  
    plt.plot(range(1, max_nro_clusters), wcss)
    plt.title('O Metodo Elbow')
    plt.xlabel('Numero de Clusters')
    plt.ylabel('WSS') #within cluster sum of squares
simulacao = metodo_elbow(15, colunas_clustering)

In [ ]:
!pip install kmodes

In [ ]:
import numpy as np
from kmodes.kmodes import KModes

# random categorical data
data = complex_sist_fim

km = KModes(n_clusters=6, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(data)

# Print the cluster centroids
print(km.cluster_centroids_)

In [ ]:
centroides = km.cluster_centroids_
centroides

In [ ]:
pd.DataFrame(centroides)

In [ ]:
%%time
#--------------------------------------------------------------------------------------#
# 1) Roda o DBSCAN
#--------------------------------------------------------------------------------------#
from sklearn.cluster import DBSCAN
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def pipeline_DBSCAN (base):
    
    # Standarize features
    scaler = StandardScaler()
    X_std = scaler.fit_transform(base)
    
    #     
    dbsc = DBSCAN(eps = .1, min_samples = 10).fit(X)
    labels = dbsc.labels_
    core_samples = np.zeros_like(labels, dtype = bool)
    core_samples[dbsc.core_sample_indices_] = True

    cluster_found_db = pd.Series(labels, name='cluster')
    df0_ = df0.set_index(cluster_found_db, append=True )
    df0_.head()

    print(f'Clusters DBScan: {cluster_found_db.unique()}')